In [2]:
import pandas as pd
import numpy as np

In [3]:
train_x = pd.read_csv("C:/Users/kxj133/Desktop/AV2/train.csv")
test = pd.read_csv("C:/Users/kxj133/Desktop/AV2/test.csv")

train_x['train'] = "Yes"
test['train'] = "No"
test['redemption_status'] = np.nan

train = pd.concat([train_x,test],axis =0,sort = False)

In [470]:
campaign_data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/campaign_data.csv")
coupon_item_mapping = pd.read_csv("C:/Users/kxj133/Desktop/AV2/coupon_item_mapping.csv")
customer_demographics = pd.read_csv("C:/Users/kxj133/Desktop/AV2/customer_demographics.csv")
customer_transaction_data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/customer_transaction_data.csv")
item_data =  pd.read_csv("C:/Users/kxj133/Desktop/AV2/item_data.csv")

In [ ]:
customer_demographics['']

In [55]:
coupon_item_mapping.shape

(92663, 2)

In [46]:
train[(train['customer_id']==1) & (train['campaign_id'] == 8)].head(1)

,id,campaign_id,coupon_id,customer_id,redemption_status,train
3062,5085,8,424,1,0.0,Yes


In [215]:
customer_demographics.shape

(760, 7)

In [471]:
customer_demographics['family_size'].value_counts()

2     303
1     248
3     104
5+     57
4      48
Name: family_size, dtype: int64

In [316]:
customer_demographics.columns

Index(['customer_id', 'age_range', 'marital_status', 'rented', 'family_size',
       'no_of_children', 'income_bracket', 'marital_status '],
      dtype='object')

In [ ]:
customer_demographics.apply(lambda x : if x['marital_status'])

In [450]:
customer_demographics.tail()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,adj_marital_status
755,1577,36-45,NaN,0,2,NaN,5,Married
756,1578,46-55,Married,0,3,1,6,Married
757,1579,46-55,NaN,0,1,NaN,4,Single
758,1580,26-35,NaN,0,2,NaN,5,Married
759,1581,26-35,NaN,0,3,1,1,Married


In [478]:
customer_demographics['adj_marital_status'].isnull().sum()

KeyError: 'adj_marital_status'

In [451]:
customer_demographics['adj_marital_status'] = customer_demographics['marital_status'].where((customer_demographics['marital_status'] != np.nan) & (customer_demographics['family_size'] != '1'),"Single")

In [454]:
customer_demographics['adj_marital_status'] = customer_demographics['adj_marital_status'].where( (customer_demographics['adj_marital_status'] != np.nan) & (~(customer_demographics['family_size']).isin(['2','3','4','5+'])),"Married")

In [410]:
customer_demographics['adj_marital_status'].isnull().sum()

146

In [412]:
customer_demographics['customer_id'].nunique()

760

In [411]:
customer_demographics['marital_status'].isnull().sum()

329

In [ ]:
customer_demographics.where()

In [358]:
customer_demographics['marital_status'] == np.nan

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
730    False
731    False
732    False
733    False
734    False
735    False
736    False
737    False
738    False
739    False
740    False
741    False
742    False
743    False
744    False
745    False
746    False
747    False
748    False
749    False
750    False
751    False
752    False
753    False
754    False
755    False
756    False
757    False
758    False
759    False
Name: marital_status, Length: 760, dtype: bool

In [327]:
df.groupby('node_i')['value_j'].transform('mean').fillna(1)
np.where(df['value_j'].notnull(),df['value_j'],values).astype(int)

array([33, 51, 69, 89, 89,  1,  1])

In [211]:
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [236]:
#### Coupon_varaibles

merged_coupon_items = pd.merge(coupon_item_mapping,item_data,how = "left",on = "item_id")

#### BRAND TYPE VARIABLE
brand_type_counts = merged_coupon_items[['coupon_id','brand_type','brand']].groupby(['coupon_id','brand_type']).count().reset_index().rename( columns = {'brand' : "brand_type_counts"})

merged_coupon_items['brand_counts'] = merged_coupon_items['coupon_id'].map(brand_type_counts.groupby(['coupon_id'])['brand_type'].count())

merged_coupon_items['only_brands'] = merged_coupon_items.apply(lambda x : 'local' if (x['brand_type'] == 'Local') and (x['brand_counts'] == 1) else("established" if (x['brand_type'] == 'Established') and (x['brand_counts'] == 1) else "both"),axis =1)

### ITEM CATEGORY
merged_coupon_items['category'].value_counts()

map_object = {'Grocery' : 'Grocery' , 'Pharmaceutical' : 'Pharmacy', 'Natural Products' : 'Natural Products','Meat' : 'Meat','Packaged Meat' : 'Meat', 'Skin & Hair Care' : 'skin care', 'Seafood' : 'seafood','Flowers & Plants' : 'Flowers&plants','Dairy, Juices & Snacks' : 'dairy&juices' }

merged_coupon_items['category_items'] = merged_coupon_items['category'].map(map_object).fillna('others')

category_counts = merged_coupon_items[['coupon_id','category_items','category']].groupby(['coupon_id','category_items']).count().reset_index()

merged_coupon_items['category_counts'] = merged_coupon_items['coupon_id'].map(category_counts.groupby(['coupon_id'])['category'].count())


merged_coupon_items.drop(['item_id','brand','brand_type','category'],axis = 1 , inplace = True)

category_list = merged_coupon_items['category_items'].unique().tolist()
merged_coupon_items['category_items_list'] = merged_coupon_items['coupon_id'].map(merged_coupon_items.groupby('coupon_id')['category_items'].apply(list))

merged_coupon_items.drop_duplicates(subset= ['coupon_id','only_brands'],inplace = True)

for  i in  category_list:
    merged_coupon_items['order_category_' + i] = merged_coupon_items['category_items_list'].apply(lambda x : 1 if i in x else 0)

merged_coupon_items[merged_coupon_items['category_counts'] == 4].head(1)

merged_coupon_items.drop('category_items_list',axis = 1, inplace = True)

In [401]:
####Customer Transaction data

customer_transaction_data['actual_price'] = customer_transaction_data["selling_price"] + (customer_transaction_data['other_discount'] * (-1)) + (customer_transaction_data['coupon_discount'] * (-1)) 
customer_transaction_data['diff_actual_selling_price'] = customer_transaction_data['actual_price']- customer_transaction_data['selling_price']
customer_transaction_data['ratio_actual_selling_price'] = customer_transaction_data['selling_price']/customer_transaction_data['actual_price'] 
###customer_transaction_data aggregation
#customer_transaction_data.groupby('customer_id')['selling_price'].sum()

max_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','selling_price'].max()
max_purchase = max_purchase.rename(columns = {'item_id' : 'max_sell_price_item',"selling_price" : 'max_sell_price'})
max_actual_price_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','actual_price'].max()
max_actual_price_purchase = max_actual_price_purchase.rename(columns = {'item_id' : 'max_actual_price_item',"actual_price" : 'max_actual_price'})

max_actual_max_sell = pd.concat([max_actual_price_purchase,max_purchase],axis =1)

max_actual_max_sell = max_actual_max_sell.reset_index()
max_actual_max_sell.drop(['max_actual_price_item'],axis =1,inplace = True)

max_actual_max_sell['max_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].max())
max_actual_max_sell['min_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].min())
max_actual_max_sell['max_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].max())
max_actual_max_sell['min_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].min())

max_actual_max_sell['diff_max_actual_max_sell_price'] = max_actual_max_sell['max_actual_price'] - max_actual_max_sell['max_sell_price']
max_actual_max_sell['ratio_max_actual_max_sell_price'] = max_actual_max_sell['max_sell_price']/max_actual_max_sell['max_actual_price']

max_actual_max_sell['purchase_count'] = max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].count())
max_actual_max_sell['coupon_usage_count'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].count()).fillna(0)
max_actual_max_sell['ratio_purchase_count_discount_count'] = max_actual_max_sell['coupon_usage_count']/max_actual_max_sell['purchase_count']
max_actual_max_sell['coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].sum()).fillna(0)
max_actual_max_sell['max_coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].min()).fillna(0)

####in which transaction he used his first coupon
coupon_first_day = customer_transaction_data[["customer_id","date","coupon_discount"]].groupby(['customer_id','date'],sort = True).sum().reset_index().groupby(['customer_id']).head(1).reset_index()
coupon_first_day.drop(['index','date'],axis =1,inplace = True)

max_actual_max_sell = pd.merge(max_actual_max_sell,coupon_first_day,how = "left",on = "customer_id")

### Since the min_diff_actual_sell_price and max_ratio_actual_selling_price is same for all the customers we are removing that that variable
max_actual_max_sell.drop(['min_diff_actual_sell_price','max_ratio_actual_selling_price'],axis =1,inplace = True)

In [402]:
coupon_item_mapping['item_list'] =  coupon_item_mapping['coupon_id'].map(coupon_item_mapping.groupby(["coupon_id"])["item_id"].apply(list))

coupon_item_mapping.drop(['item_id'],axis =1,inplace = True)

coupon_item_mapping.drop_duplicates(subset = ['coupon_id'],inplace= True)

In [442]:
merged_data = pd.merge(train,campaign_data,how = "left",on = "campaign_id" )

merged_data = pd.merge(merged_data,coupon_item_mapping,how = "left",on = "coupon_id")

merged_data = pd.merge(merged_data,customer_demographics,how = "left" , on = "customer_id")

In [443]:
## transaction data
merged_data['total_purchase_price'] = merged_data['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].sum())
merged_data = pd.merge(merged_data,max_purchase,how = "left", on = "customer_id")

In [444]:
merged_data['len_item_list'] = merged_data['item_list'].apply(lambda x : len(x))

merged_data.drop('item_list',axis =1,inplace = True)

In [445]:
merged_data[merged_data['customer_id'] == 1].head(2)

,id,campaign_id,coupon_id,customer_id,train,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket,total_purchase_price,max_sell_price_item,max_sell_price,len_item_list
2815,4674,29,597,1,Yes,Y,08/10/12,30/11/12,70+,Married,0.0,2,NaN,4.0,98513.93,73348,445.25,63
3062,5085,8,424,1,Yes,X,16/02/13,05/04/13,70+,Married,0.0,2,NaN,4.0,98513.93,73348,445.25,3


In [446]:
merged_data.drop(['age_range','marital_status','rented','family_size','no_of_children','income_bracket'],axis =1,inplace = True)

In [447]:
merged_data = pd.merge(merged_data,max_actual_max_sell,how = "left", on = "customer_id",sort = True)

In [282]:
merged_data['start_date'] = pd.to_datetime(merged_data['start_date'],format = "%d/%m/%y")

merged_data['end_date'] = pd.to_datetime(merged_data['end_date'],format = "%d/%m/%y")

NameError: name 'merged_data' is not defined

In [485]:
merged_data['campaign_days'] = merged_data['end_date'] - merged_data['start_date']

In [288]:
from datetime import date
start=  pd.to_datetime('03/12/13',format = "%d/%m/%y")
end =  pd.to_datetime('03/10/13',format = "%d/%m/%y")
abs((start -end).days)

61

In [499]:
merged_data['start_date'].dt.quarter

0         4
1         1
2         1
3         4
4         2
5         1
6         4
7         1
8         2
9         2
10        4
11        1
12        2
13        4
14        2
15        4
16        4
17        2
18        4
19        2
20        1
21        2
22        4
23        4
24        4
25        1
26        1
27        1
28        2
29        4
         ..
128565    3
128566    3
128567    3
128568    3
128569    3
128570    3
128571    3
128572    3
128573    3
128574    3
128575    3
128576    3
128577    3
128578    3
128579    3
128580    3
128581    3
128582    3
128583    3
128584    3
128585    3
128586    3
128587    3
128588    3
128589    3
128590    3
128591    3
128592    3
128593    3
128594    3
Name: start_date, Length: 128595, dtype: int64

In [501]:
merged_data['start_date_month_end'] = merged_data['start_date'].dt.is_month_end
merged_data['end_date_month_end'] = merged_data['end_date'].dt.is_month_end
merged_data['start_date_year_end'] = merged_data['start_date'].dt.is_year_end
merged_data['start_date_quarter'] = merged_data['start_date'].dt.quarter

In [506]:
drop_columns = ['campaign_id','coupon_id','start_date','end_date']
merged_data.drop(drop_columns, axis = 1,inplace = True)

In [508]:
merged_data.head(2)

,id,customer_id,train,campaign_type,total_purchase_price,max_sell_price_item_x,max_sell_price_x,len_item_list,max_actual_price,max_sell_price_item_y,...,coupon_usage_count,ratio_purchase_count_discount_count,coupon_usage_amt,max_coupon_usage_amt,coupon_discount,campaign_days,start_date_month_end,end_date_month_end,start_date_year_end,start_date_quarter
0,4674,1,Yes,Y,98513.93,73348,445.25,63,906.53,73348,...,78.0,0.074427,-2116.83,-106.86,-35.62,53 days,False,True,False,4
1,5085,1,Yes,X,98513.93,73348,445.25,3,906.53,73348,...,78.0,0.074427,-2116.83,-106.86,-35.62,48 days,False,False,False,1


In [519]:
merged_data['customer_id'].nunique()/train_data.shape[0]

0.0201865533565568

In [525]:
train_data = merged_data.loc[merged_data['train'] == 'Yes']
test_data = merged_data.loc[merged_data['train'] == 'No']

In [523]:
data = pd.read_csv("C:/Users/kxj133/Desktop/AV2/train.csv")

In [524]:
data.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status'], dtype='object')

In [543]:
final_data = pd.merge(train_data,data[['id','redemption_status']], how = 'left',on = 'id')

In [544]:
final_data.head(2)

,id,customer_id,train,campaign_type,total_purchase_price,max_sell_price_item_x,max_sell_price_x,len_item_list,max_actual_price,max_sell_price_item_y,...,ratio_purchase_count_discount_count,coupon_usage_amt,max_coupon_usage_amt,coupon_discount,campaign_days,start_date_month_end,end_date_month_end,start_date_year_end,start_date_quarter,redemption_status
0,4674,1,Yes,Y,98513.93,73348,445.25,63,906.53,73348,...,0.074427,-2116.83,-106.86,-35.62,53 days,False,True,False,4,0
1,5085,1,Yes,X,98513.93,73348,445.25,3,906.53,73348,...,0.074427,-2116.83,-106.86,-35.62,48 days,False,False,False,1,0


In [545]:
final_data.drop(['train','customer_id','id','campaign_days'],axis =1,inplace = True)

In [546]:
le = LabelEncoder()
final_data['campaign_type'] = le.fit_transform(final_data['campaign_type'])